In [ ]:
/*** ONE TIME ONLY ***/
use role accountadmin;
use database customer_support;
use schema support;
use warehouse COMPUTE_WH;


CREATE OR REPLACE CORTEX SEARCH SERVICE customer_master_data_search_svc
  ON full_details
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  AS (
   select full_details
   from customer_master_data
);




In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
from snowflake.core import Root

root = Root(session)

# query service
svc = (root
  .databases["customer_support"]
  .schemas["support"]
  .cortex_search_services["customer_master_data_search_svc"]
)



In [ ]:
resp = svc.search(
  query="Patton 887 Aaron Center   WY 15339",
  columns=["full_details"],
  limit=1
).to_json()

st.info(resp)

In [ ]:
import json
new_data = session.table("customer_support.support.new_customer_data").select("FULL_DETAILS").to_pandas()
df = pd.DataFrame(columns=['New Address', 'Matching Exsiting address'])
for index, row in new_data.iterrows():
    resp = svc.search(
        query=row["FULL_DETAILS"],
        columns=["full_details"],
        limit=1
    ).to_json()
    j = json.loads(resp)
    df.loc[index] = [ row["FULL_DETAILS"], j["results"][0]["full_details"]]
st.write(df)